In [1]:
from PyPDF2 import PdfReader
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# nltk.download('stopwords')
# nltk.download('punkt')

In [32]:
Dataset = "./Dataset"
Resume_dir = os.path.join(Dataset,"Resumes")
Job_dir = os.path.join(Dataset,"JobDescription")
resumedataset = []
jobdataset = []
resume = ''
job =''
jobs = []

In [33]:
for file in os.listdir(Resume_dir):
    reader = PdfReader(os.path.join(Resume_dir,file))
    # print(os.path.join(Resume_dir,file))
    number_of_pages = len(reader.pages)
    for i in range(number_of_pages):
        page = reader.pages[i]
        text = page.extract_text()
        resume += text
    resumedataset += [resume]
    resume = ''
    i = 0
resumedataset[1]

./Dataset/Resumes/john_doe.pdf
./Dataset/Resumes/barry_allen_fe.pdf
./Dataset/Resumes/harvey_dent_mle.pdf
./Dataset/Resumes/bruce_wayne_fullstack.pdf
./Dataset/Resumes/alfred_pennyworth_pm.pdf


'Barry Allen\nFront-End DeveloperGoogle HQ, Mountain View, CA, USA\n♂¶obile-alt(123) 456-7890\n/envel⌢peallthe.stars@google.com\n/linkedin-inbwayne\n/githubbwayne\nObjective\nSeeking a challenging role as a Front-End Developer where I can leverage my knowledge of UI/UX\ndesign and modern web technologies to create intuitive and engaging user interfaces.\nEducation\n2018 – 2022 B.Tech, Computer Science and Engineering ,Indian Institute of Technology\nDelhi , New Delhi, India\nOverall GPA: 9.5/10\nSkills\nJavaScript (ES6+), TypeScript, HTML5, CSS3, Python, React, Redux, Angular,\nVue.js, Node.js, Express.js, D3.js, Git, Docker, Webpack, Babel, Google Cloud\nPlatform, Firebase, RESTful APIs, GraphQL, Agile Development, Test-Driven\nDevelopment, Responsive Design, UI/UX\nExperience\nJune 2022 –\nPresentSoftware Engineer (Front-End) ,Google , Mountain View, CA, USA\nDeveloping intuitive and engaging user interfaces using React and Redux. Working closely\nwith UX designers to implement respo

In [34]:
for file in os.listdir(Job_dir):
    reader = PdfReader(os.path.join(Job_dir,file))
    jobs.append(os.path.splitext(file)[0])
    number_of_pages = len(reader.pages)
    for i in range(number_of_pages):
        page = reader.pages[i]
        text = page.extract_text()
        job += text
    jobdataset.append(job)

In [35]:
print(jobs)

['job_desc_java_developer', 'job_desc_full_stack_engineer', 'job_desc_front_end_engineer', 'job_desc_product_manager']


In [36]:
# Define multiple universities names patterns
sub_patterns = [
    
                '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* at [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]*',
    
                '[A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* at [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* University of [A-Z][a-z]*',
    
                '[A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]*',
                '[A-Z][a-z]* University of [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* University of [A-Z][a-z]* at [A-Z][a-z]*',
                '[A-Z][a-z]* University of [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* University of [A-Z][a-z]*',
    
                'University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                'University of [A-Z][a-z]* [A-Z][a-z]* at [A-Z][a-z]*',
                'University of [A-Z][a-z]* at [A-Z][a-z]* [A-Z][a-z]*',
                'University of [A-Z][a-z]* at [A-Z][a-z]*',
                'University of [A-Z][a-z]* [A-Z][a-z]*',
                'University of [A-Z][a-z]*',
    
                '[A-Z][a-z]* [A-Z][a-z]* University',
                '[A-Z][a-z]* University',
                '[A-Z]* University',
    
                '[A-Z][a-z]* Institute of [A-Z][a-z]* & [A-Z][a-z]*',
                '[A-Z][a-z]* Institute of [A-Z][a-z]* [A-Z][a-z]*',
                '[A-Z][a-z]* Institute of [A-Z][a-z]*',
    
                '[A-Z][a-z]* [A-Z][a-z]* Community College',
                '[A-Z][a-z]* Community College',
    
                'College of [A-Z][a-z]* & [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* College of [A-Z][a-z]*',
                '[A-Z]* College of [A-Z][a-z]*',
                '[A-Z][a-z]* College of [A-Z][a-z]*',
                '[A-Z][a-z]* [A-Z][a-z]* College',
                '[A-Z][a-z]* College',
    
                '[A-Z][a-z]* [A-Z][a-z]* High School',
                '[A-Z][a-z]* High School',
    
                '[A-Z][a-z]* [A-Z][a-z]* Military Academy',
                '[A-Z][a-z]* Military Academy',
    
                'Universidad de los [A-Z][a-z]*',
                'Universidad de las [A-Z][a-z]*',
                'Universidad de [A-Z][a-z]*',

                'Universidad [A-Z][a-z]* de [A-Z][a-z]* [A-Z][a-z]*',
                'Universidad [A-Z][a-z]* de los [A-Z][a-z]*',
                'Universidad [A-Z][a-z]* de las [A-Z][a-z]*',
                'Universidad [A-Z][a-z]* de [A-Z][a-z]*',
                ]

In [37]:
# Join university patterns
university_patterns = '({})'.format('|'.join(sub_patterns))

In [38]:
class Document:
    def __init__(self, text):
        
        self.text = text
        self.firstName = ""
        self.lastName = ""
        self.emailAddress = ""
        self.universities = []
        
    def get_first_name(self):

        firstWords = self.text.split()[:10]
        for word in firstWords:
            if word.isdigit() == False and word[0].isupper() and word.lower() not in ["page", "last", "updated"] and len(word) > 1:
                self.firstName = word
                break
                
    def get_last_name(self):

        firstWords = self.text.split()[self.text.split().index(self.firstName)+1:10]
        for word in firstWords:
            if word.isdigit() == False and word[0].isupper() and word.lower() not in ["page", "last", "updated"] and len(word) > 1:
                self.lastName = word
                break
                
    def get_email(self):
        
        match = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', self.text)
        self.emailAddress = match.group(0)
        
    def get_univerisities(self):
        
        university_matches = re.findall(university_patterns, self.text)
        universities_to_remove = ["University","University "," University"," University ","College"]
        self.universities = list(set(university_matches))
        
        for university in universities_to_remove:
            try:
                self.universities.remove(university)
            except:
                pass
    def preprocess_text(self):
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(self.text.lower())
        filtered_tokens = [word for word in word_tokens if word.isalpha() and word not in stop_words]
        return filtered_tokens

In [39]:
# Instantiate myResume variable
# myResume = Document(resumedataset[0])
# myJD = Document(jobdataset[0])

# Obtain resume information
# myResume.get_first_name()
# myResume.get_last_name()
# myResume.get_email()
# myResume.get_univerisities()
# myResume.preprocess_text()

In [40]:
# Print results
# print(f"First name: {myResume.firstName}")
# print(f"Last name: {myResume.lastName}")
# print(f"Email address: {myResume.emailAddress}")
# print(f"Universities: {myResume.universities}")

In [41]:
def calculate_similarity(resume_text, jd_text):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([resume_text, jd_text])
        similarity_score = cosine_similarity(vectors)[0][1]
        return similarity_score


In [51]:
def main():
    best_score = 0
    candidate_full_name = ''
    print(len(resumedataset))
    i = 0
    print("what are you looking for out of these jobs, choose by number",jobs)
    while True:
        choice_of_job = input()
        if choice_of_job.isnumeric():
            break
        else:
            print("Invalid input, please enter a number")
    myJD = Document(jobdataset[int(choice_of_job)-1])
    preprocessed_jd_text = myJD.preprocess_text()
    for i in range(len(resumedataset)):
        resume = Document(resumedataset[i])
        resume.get_first_name()
        resume.get_last_name()
        preprocessed_resume_text = resume.preprocess_text()
        similarity_score = calculate_similarity(' '.join(preprocessed_resume_text), ' '.join(preprocessed_jd_text))
        if best_score < similarity_score:
            best_score = similarity_score
            candidate_full_name = resume.firstName + " " + resume.lastName
    print("The best candidate for this job is: ",candidate_full_name,"\nTheir score is: ",best_score)
    # print("The similarity score between the resume and job description is: ", similarity_score)

if __name__ == "__main__":
    main()

5
what are you looking for out of these jobs, choose by number ['job_desc_java_developer', 'job_desc_full_stack_engineer', 'job_desc_front_end_engineer', 'job_desc_product_manager']
0.2926270794382915
JOHN
0.1514142953927146
Barry
0.11465514901308495
Harvey
0.24695640830448673
Bruce
0.31396393014440854
Alfred
The best candidate for this job is:  Alfred Pennyworth 
Their score is:  0.31396393014440854
